# Prediction 

## Introduction

In labor economics an important question is what determines the wage of workers. This is a causal question, but we could begin to investigate from a predictive perspective.

In the following wage example,  𝑌  is the hourly wage of a worker and  𝑋  is a vector of worker's characteristics, e.g., education, experience, sex. Two main questions here are:

* How to use job-relevant characteristics, such as education and experience, to best predict wages?

* What is the difference in predicted wages between male and female workers with the same job-relevant characteristics?

In this lab, we focus on the prediction question first.

## Data

The data set we consider is from the March Supplement of the U.S. Current Population Survey, year 2015. We select white non-hispanic individuals, aged 25 to 64 years, and working more than 35 hours per week during at least 50 weeks of the year. We exclude self-employed workers; individuals living in group quarters; individuals in the military, agricultural or private household sectors; individuals with inconsistent reports on earnings and employment status; individuals with allocated or missing information in any of the variables used in the analysis; and individuals with hourly wage below  3 .

The variable of interest  𝑌  is the hourly wage rate constructed as the ratio of the annual earnings to the total number of hours worked, which is constructed in turn as the product of number of weeks worked and the usual number of hours worked per week. In our analysis, we also focus on single (never married) workers. The final sample is of size  $n=5150$ .

In [ ]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import sklearn.linear_model as lm
import sklearn.metrics as metrics
import statsmodels.formula.api as smf
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.metrics import r2_score
import warnings
# ignore potential convergence warnings; for some small penalty levels, tried out, optimization might not converge
warnings.simplefilter('ignore')

## Data Analysis

In [ ]:
file = "https://raw.githubusercontent.com/CausalAIBook/MetricsMLNotebooks/main/data/wage2015_subsample_inference.csv"
df = pd.read_csv(file)

In [ ]:
df.describe()

### Construct variables

We are constructing the output variable  $Y$  and the matrix  $Z$  which includes the characteristics of workers that are given in the data.

In [ ]:
Y = np.log(df['wage'])
Z = df.drop(['wage', 'lwage'], axis=1)
Z.shape

For the outcome variable wage and a subset of the raw regressors, we calculate the empirical mean and other empirical measures to get familiar with the data.



In [ ]:
Z.describe()

E.g., the share of female workers in our sample is ~44% ( 𝑠𝑒𝑥=1  if female).

In [ ]:
# if you want to print this table to latex
print(Z.describe().style.to_latex())

## Prediction Question

Now, we will construct a prediction rule for hourly wage  $Y$, which depends linearly on job-relevant characteristics  $X$:

$$
𝑌=\beta′𝑋+𝜖.
$$

Our goals are:

* Predict wages using various characteristics of workers.

* Assess the predictive performance using the (adjusted) sample MSE, the (adjusted) sample $R^2$  and the out-of-sample MSE and  $R^2$ .

We employ two different specifications for prediction:

1. **Basic Model**:  $𝑋$  consists of a set of raw regressors (e.g. sex, experience, education indicators, occupation and industry indicators, regional indicators).
2. **Flexible Model**:  $𝑋$  consists of all raw regressors from the basic model plus occupation and industry indicators, transformations (e.g.,  $exp^2$  and  $exp^3$ ) and additional two-way interactions of polynomial in experience with other regressors. An example of a regressor created through a two-way interaction is experience times the indicator of having a college degree.

Using the **Flexible Model**, enables us to approximate the real relationship by a more complex regression model and therefore to reduce the bias. The **Flexible Model** increases the range of potential shapes of the estimated regression function. In general, flexible models often deliver good prediction accuracy but give models which are harder to interpret.

Now, let us fit both models to our data by running ordinary least squares (ols):

In [ ]:
# 1. Basic Model
model_base = 'lwage ~ sex + exp1 + shs + hsg+ scl + clg + mw + so + we + C(occ2) + C(ind2)'
base = smf.ols(model_base, data=df)
results_base = base.fit()

In [ ]:
rsquared_base = results_base.rsquared
rsquared_adj_base = results_base.rsquared_adj
mse_base = np.mean(results_base.resid**2)
mse_adj_base = results_base.mse_resid
print(f'Rsquared={rsquared_base:.4f}')
print(f'Rsquared_adjusted={rsquared_adj_base:.4f}')
print(f'MSE={mse_base:.4f}')
print(f'MSE_adjusted={mse_adj_base:.4f}')

In [ ]:
# verify the formulas
X, y = base.data.exog, base.data.endog
n, p = X.shape
mse = np.mean((y - results_base.predict(X, transform=False))**2)
mse_adj = mse * n / (n - p)
rsquared = 1 - mse / np.var(y)
rsquared_adj = 1 - mse_adj / np.var(y)
print(f'Rsquared={rsquared:.4f}')
print(f'Rsquared_adjusted={rsquared_adj:.4f}')
print(f'MSE={mse:.4f}')
print(f'MSE_adjusted={mse_adj:.4f}')

In [ ]:
# 2. Flexible Model
model_flex = ('lwage ~ sex + shs+hsg+scl+clg+C(occ2)+C(ind2)+mw+so+we '
                      '+ (exp1+exp2+exp3+exp4)*(shs+hsg+scl+clg+C(occ2)+C(ind2)+mw+so+we)')
flex = smf.ols(model_flex, data=df)
results_flex = flex.fit()

In [ ]:
rsquared_flex = results_flex.rsquared
rsquared_adj_flex = results_flex.rsquared_adj
mse_flex = np.mean(results_flex.resid**2)
mse_adj_flex = results_flex.mse_resid
print(f'Rsquared={rsquared_flex:.4f}')
print(f'Rsquared_adjusted={rsquared_adj_flex:.4f}')
print(f'MSE={mse_flex:.4f}')
print(f'MSE_adjusted={mse_adj_flex:.4f}')

#### Next we try the Lasso

We use the statsmodels package with the formula api for uniformity in feature construction and the sklearn Lasso with cross-validation to tune the regularization hyperparameter.

In [ ]:
# Lasso with cross-validation
X = flex.data.exog[:, 1:] # exclude the intercept; we don't want the lasso to penalize the intercept
y = flex.data.endog

# train model using Lasso with cross validation and variable normalization
lasso = Pipeline([('scale', StandardScaler()), # standardize the variables
                  ('lasso', lm.LassoCV())])
lasso.fit(X, y)

In [ ]:
# verify the formulas
n, p = X.shape
p += 1
mse_lasso = np.mean((y - lasso.predict(X))**2)
mse_adj_lasso = mse_lasso * n / (n - p)
rsquared_lasso = 1 - mse_lasso / np.var(y)
rsquared_adj_lasso = 1 - mse_adj_lasso / np.var(y)
print(f'Rsquared={rsquared_lasso:.4f}')
print(f'Rsquared_adjusted={rsquared_adj_lasso:.4f}')
print(f'MSE={mse_lasso:.4f}')
print(f'MSE_adjusted={mse_adj_lasso:.4f}')

In [ ]:
# store the results in a table
res_df = pd.DataFrame()

res_df['Model'] = ['Basic reg', 'Flexible reg', 'Flexible Lasso']

res_df['p'] = [results_base.params.shape[0],
               results_flex.params.shape[0],
               results_flex.params.shape[0]]

res_df['R2'] = [rsquared_base, rsquared_flex, rsquared_lasso]
res_df['MSE'] = [mse_base, mse_flex, mse_lasso]

res_df['adj_R2'] = [rsquared_adj_base, rsquared_adj_flex, rsquared_adj_lasso]
res_df['adj_MSE'] = [mse_adj_base, mse_adj_flex, mse_adj_lasso]

# Show results
res_df.head()

In [ ]:
# print to Latex
print(res_df.style.to_latex())

## Data Splitting

Measure the prediction quality of the two models via data splitting:

* Randomly split the data into one training sample and one testing sample. Here we just use a simple method (stratified splitting is a more sophiscticated version of splitting that we can consider).
* Use the training sample for estimating the parameters of the Basic Model and the Flexible Model.
* Use the testing sample for evaluation. Predict the  𝚠𝚊𝚐𝚎  of every observation in the testing sample based on the estimated parameters in the training sample.
* Calculate the Mean Squared Prediction Error $MSE_{test}$ based on the testing sample for both prediction models.

In [ ]:
# split the data into training and testing sets

# get the indices
n = df.shape[0]
train_idx, test_idx = train_test_split(np.arange(n), test_size=.2, shuffle=True)

In [ ]:
# Basic model

# estimating the parameters in the training sample
mod1 = smf.ols(model_base, data=df)
X1 = mod1.data.exog
y1 = mod1.data.endog

# separate training and testing sets
X1_train = X1[train_idx,:]
X1_test = X1[test_idx,:]
y1_train = y1[train_idx]
y1_test = y1[test_idx]

# estimating the parameters in the training sample
regbasic = sm.OLS(y1_train, X1_train).fit()

# predict out of sample
trainregbasic = regbasic.predict(X1_test)

# calculating out-of-sample MSE
MSE_test1 = sum((y1_test-trainregbasic)**2)/y1_test.shape[0]
R2_test1 = 1. - MSE_test1/np.var(y1_test)

print("Test MSE for the basic model: "+ str(MSE_test1))
print("Test R2 for the basic model: "+ str(R2_test1))

In the basic model, the  $𝑀𝑆𝐸_{𝑡𝑒𝑠𝑡}$  is quite closed to the  $𝑀𝑆𝐸_{𝑠𝑎𝑚𝑝𝑙𝑒}$.

In [ ]:
# Flexible model

# estimating the parameters in the training sample
mod2 = smf.ols(model_flex, data=df)
X2 = mod2.data.exog
y2 = mod2.data.endog

# separate training and testing sets
X2_train = X2[train_idx,:]
X2_test = X2[test_idx,:]
y2_train = y2[train_idx]
y2_test = y2[test_idx]

# estimating the parameters in the training sample
regflex = sm.OLS(y2_train, X2_train).fit()

# predict out of sample
trainregflex = regflex.predict(X2_test)

# calculating out-of-sample MSE
MSE_test2 = np.mean((y2_test - trainregflex)**2)
R2_test2 = 1. - MSE_test2 / np.var(y2_test)

print("Test MSE for the flexible model: "+ str(MSE_test2))
print("Test R2 for the flexible model: "+ str(R2_test2))

In the flexible model, the discrepancy between the  $𝑀𝑆𝐸_{𝑡𝑒𝑠𝑡}$  and the  $𝑀𝑆𝐸_{𝑠𝑎𝑚𝑝𝑙𝑒}$  is not large.

It is worth to notice that the  $𝑀𝑆𝐸_{𝑡𝑒𝑠𝑡}$  vary across different data splits. Hence, it is a good idea average the out-of-sample MSE over different data splits to get valid results.

Nevertheless, we observe that, based on the out-of-sample  $𝑀𝑆𝐸$ , the basic model using ols regression performs is about as well (or slightly better) than the flexible model.

Next, let us use lasso regression in the flexible model instead of ols regression. Lasso (least absolute shrinkage and selection operator) is a penalized regression method that can be used to reduce the complexity of a regression model when the number of regressors  $p$  is relatively large in relation to  $n$ .

Note that the out-of-sample  𝑀𝑆𝐸  on the test sample can be computed for any other black-box prediction method as well. Thus, let us finally compare the performance of lasso regression in the flexible model to ols regression.

In [ ]:
# train model using Lasso with cross validation and variable normalization
lasso = Pipeline([('scale', StandardScaler()), # standardize the variables
                  ('lasso', lm.LassoCV())])
lasso.fit(X2_train[:, 1:], y2_train)

# predict out of sample
trainreglasso = lasso.predict(X2_test[:, 1:])

# calculating out-of-sample MSE
MSE_test3 = np.mean((y2_test - trainreglasso)**2)
R2_test3 = 1. - MSE_test3 / np.var(y2_test)

print("Test MSE for the basic model: "+ str(MSE_test3))
print("Test R2 for the basic model: "+ str(R2_test3))

In [ ]:
# store the results in a table
res_df2 = pd.DataFrame()

res_df2['Model'] = ['Basic reg', 'Flexible reg', 'Flexible Lasso']

res_df2['$MSE_{test}$'] = [MSE_test1, MSE_test2, MSE_test3]
res_df2['$R^2_{test}$'] = [R2_test1, R2_test2, R2_test3]

# Show results
res_df2.head()

In [ ]:
# print to Latex
print(res_df2.style.to_latex())